In [69]:
import plotly.graph_objects as go
import networkx as nx
import os

# reading input log file

# test_name = "ddmd"
# test_name = "vist"
# test_name = "s9f9p8"
# test_name = "f24s9p12_1"
# test_name = "ss_f24s9p12" # mapping to workload_name=pyflextrkr
# test_name = "f48s9p24_1" # mapping to workload_name=pyflextrkr
# test_name = "h5bench-sync-write-3d-contig-contig-read-full-80g_3"
# test_name = "64_ds200_p1_r32_t3"

test_name = "wrf_f24s9p12"

# test_name = "strong_scale"
WORKLOAD_NAME = "pyflextrkr" # choices: pyflextrkr --> f48s9p24, h5bench --> 

stat_path=f"example_stat/{test_name}"
image_path=f"{stat_path}/images"

VFD_ACCESS_SKIP=10

ADD_ADDR=False


In [70]:
# My utility functions
import utils.stat_loader as sload
import utils.stat_print as sp
import utils.vfd_stat2graph as vfd2g
import utils.vfd_graph2sankey as vfd2sk
import utils.full_stat2graph as f2g

# Improve Functions
## TODO
- Add dataset nodes
- Add file address ordering nodes


In [71]:
STAGE_START = 0
STAGE_END = 10


TASK_ORDER_LIST = sload.load_task_order_list(stat_path)
print(TASK_ORDER_LIST)

STAGE_END = sload.correct_end_stage(TASK_ORDER_LIST, STAGE_END)
TASK_ORDER_LIST = sload.current_task_order_list(TASK_ORDER_LIST, STAGE_START, STAGE_END)

TASK_LISTS = list(TASK_ORDER_LIST.keys())

print(f"TASK_ORDER_LIST = {TASK_ORDER_LIST}")
TASK_LISTS


{'run_idfeature': 0, 'run_tracksingle': 1, 'run_gettracks': 2, 'run_trackstats': 3, 'run_identifymcs': 4, 'run_matchpf': 5, 'run_robustmcs': 6, 'run_mapfeature': 7, 'run_speed': 8}
STAGE_END is not in TASK_ORDER_LIST, set to max order: 8
TASK_ORDER_LIST = {'run_idfeature': 0, 'run_tracksingle': 1, 'run_gettracks': 2, 'run_trackstats': 3, 'run_identifymcs': 4, 'run_matchpf': 5, 'run_robustmcs': 6, 'run_mapfeature': 7, 'run_speed': 8}


['run_idfeature',
 'run_tracksingle',
 'run_gettracks',
 'run_trackstats',
 'run_identifymcs',
 'run_matchpf',
 'run_robustmcs',
 'run_mapfeature',
 'run_speed']

In [72]:
# Search for log file lines to get workflow time
def get_workflow_time(stat_path:str, workload_name:str=WORKLOAD_NAME):
    # Find any file with ".log" extension
    log_file = None
    for file in os.listdir(stat_path):
        if file.endswith(".log"):
            log_file = os.path.join(stat_path, file)
            break
    print(f"log_file = {log_file}")

    # Read file and search for workflow time starting with
    if workload_name == "pyflextrkr":
        TIME_STR_START = "Pyflextrkr TIME:"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    WORKFLOW_TIME = float(line.split(" ")[2])
                    # Convert from seconds to microseconds
                    WORKFLOW_TIME = WORKFLOW_TIME * 1000000
                    break
    elif workload_name == "h5bench": 
        WORKFLOW_TIME = 0
        TIME_STR_START = "Runtime:"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    runtime = float(line.split(" ")[7])
                    print(f"runtime = {runtime}")
                    # Convert from seconds to microseconds
                    WORKFLOW_TIME += runtime * 1000000
    else:
        print("WORKLOAD_NAME not recognized")
        WORKFLOW_TIME = 0

    return WORKFLOW_TIME

WORKFLOW_TIME = get_workflow_time(stat_path)
print(f"WORKFLOW_TIME = {WORKFLOW_TIME} us")

def get_workflow_stage_time(stat_path:str, workload_name:str=WORKLOAD_NAME):
    # Find any file with ".log" extension
    log_file = None
    for file in os.listdir(stat_path):
        if file.endswith(".log"):
            log_file = os.path.join(stat_path, file)
            break
    print(f"log_file = {log_file}")

    # Read file and search for workflow time starting with
    stage_time_ms = {}
    if workload_name == "pyflextrkr":
        STAGE_STR_TIME = "__main__ - INFO - Stage"
        # TIME_STR_START = "Pyflextrkr TIME:"
        with open(log_file, "r") as f:
            for line in f:
                if STAGE_STR_TIME in line:
                    # line format: "2024-04-02 01:16:01,485 - __main__ - INFO - Stage 2 [run_tracksingle]: 52605 milliseconds, 52.61 seconds"
                    line_text = line.split(" ")
                    stage_name = line_text[9].replace("[", "").replace("]:", "")
                    stage_time_ms[stage_name] = {}
                    stage_time_ms[stage_name] = float(line_text[10])
                    # stage_time[stage_name]["seconds"] = float(line_text[12])
    
    for task, time_ms in stage_time_ms.items():
        print(f"{task} = {time_ms} (ms)")
    
    # calculate the total time
    total_time = [time_ms for time_ms in stage_time_ms.values()]
    total_time = sum(total_time)
    print(f"Total time = {total_time} (ms)")
    # calculate the percentage of each stage
    for task, time_ms in stage_time_ms.items():
        stage_time_ms[task] = time_ms / total_time * 100
        print(f"{task} = {stage_time_ms[task]} %")

get_workflow_stage_time(stat_path)

log_file = example_stat/wrf_f24s9p12/run.log
WORKFLOW_TIME = 476226487.159729 us
log_file = example_stat/wrf_f24s9p12/run.log
run_idfeature = 143466.0 (ms)
run_tracksingle = 70609.0 (ms)
run_gettracks = 35776.0 (ms)
run_trackstats = 2715.0 (ms)
run_matchpf = 18416.0 (ms)
run_robustmcs = 33151.0 (ms)
run_mapfeature = 36108.0 (ms)
run_speed = 29498.0 (ms)
Total time = 369739.0 (ms)
run_idfeature = 38.80196571094745 %
run_tracksingle = 19.096984629698248 %
run_gettracks = 9.676014702262947 %
run_trackstats = 0.7343017642174615 %
run_matchpf = 4.98081078815056 %
run_robustmcs = 8.966054432991921 %
run_mapfeature = 9.765807772509797 %
run_speed = 7.978060199221614 %


In [73]:
vfd_files = sload.find_files_with_pattern(stat_path, "vfd")
# vfd_files = vfd_files[0:1]
print(f"vfd_files: {vfd_files}")

vfd_dict = sload.load_stat_json(vfd_files)
# print(vfd_dict)


vol_files = sload.find_files_with_pattern(stat_path, "vol")
# vol_files = vol_files[0:1]
print(f"vol_files: {vol_files}")

vol_dict = sload.load_stat_json(vol_files)

print("loading json done")

vfd_files: ['example_stat/wrf_f24s9p12/339598-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339586-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339570-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339590-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339583-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339555-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339787-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339578-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339559-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339574-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339507-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339567-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339753-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339783-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339562-vfd_data_stat.json', 'example_stat/wrf_f24s9p12/339594-vfd_data_stat.json']
loading example_stat/wrf_f24s9p12/339598-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339586-vfd_dat

loading example_stat/wrf_f24s9p12/339570-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339590-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339583-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339555-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339787-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339578-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339559-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339574-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339507-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339567-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339753-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339783-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339562-vfd_data_stat.json
loading example_stat/wrf_f24s9p12/339594-vfd_data_stat.json
vol_files: ['example_stat/wrf_f24s9p12/339590-vol_data_stat.json', 'example_stat/wrf_f24s9p12/339787-vol_data_stat.json', 'example_stat/wrf_f24s9p12/339555-vol_

In [74]:
# Show VFD Tracker overhead
# sp.show_all_overhead("VFD", vfd_dict)
# "VOL": ["VOL-Overhead(us)", "VOL-Init(us)", "VOL-Term(us)", "VOL-Log(us)",
#         "VOL-HT-Overhead(us)", "VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)",
#         "VOL-FILE_LL-Overhead(us)", "VOL-FILE_LL-Add(us)", "VOL-FILE_LL-Rm(us)", "VOL-FILE_LL-Update(us)",
#         "VOL-DS_LL-Overhead(us)", "VOL-DS_LL-Add(us)", "VOL-DS_LL-Rm(us)", "VOL-DS_LL-Update(us)",
#         "VOL-GRP_LL-Overhead(us)", "VOL-GRP_LL-Add(us)", "VOL-GRP_LL-Rm(us)",
#         "VOL-DT_LL-Overhead(us)", "VOL-DT_LL-Add(us)", "VOL-DT_LL-Rm(us)",
#         "VOL-All_LL-Overhead(us)"],
# ["VFD-Overhead(us)", ]
        
def show_vfd_io_time(vfd_dict):
    io_time_type = ["POSIX-READ-Time(us)", "POSIX-WRITE-Time(us)", "POSIX-OPEN-Time(us)", "POSIX-CLOSE-Time(us)", "POSIX-DELETE-Time(us)",
                "MMAP-READ-Time(us)", "MMAP-WRITE-Time(us)", "MMAP-OPEN-Time(us)", "MMAP-CLOSE-Time(us)",]


def get_first_last_file_time(entry, first_file_time, last_file_time):
    if "open_time(us)" in entry.keys():
        task_open_time = entry['open_time(us)']
        if first_file_time == -1 or task_open_time < first_file_time:
            first_file_time = task_open_time
    if "close_time(us)" in entry.keys():
        task_close_time = entry['close_time(us)']
        if last_file_time == -1 or task_close_time > last_file_time:
            last_file_time = task_close_time
    return first_file_time, last_file_time

def show_detail_overhead_old(stat_type, file_dict, task_list, workflow_time=0, workload_name="pyflextrkr"):
    overhead_dict = {
        "VOL": ["VOL-Init(us)", "VOL-Term(us)", "VOL-Log(us)",
                "VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)",
                "VOL-FILE_LL-Add(us)", "VOL-FILE_LL-Rm(us)", "VOL-FILE_LL-Update(us)",
                "VOL-DS_LL-Add(us)", "VOL-DS_LL-Rm(us)", "VOL-DS_LL-Update(us)",
                "VOL-GRP_LL-Add(us)", "VOL-GRP_LL-Rm(us)",
                "VOL-DT_LL-Add(us)", "VOL-DT_LL-Rm(us)"],

        "VFD": ["VFD-Init(us)", "VFD-Term(us)", "VFD-Tracker-Init(us)", "VFD-Stat-Add(us)", 
                "VFD-Stat-Update(us)", "VFD-Stat-Rm(us)", "VFD-Stat-Log(us)"]
    }
    
    
    overhead_time_list = []
    first_file_time = -1
    last_file_time = -1
    
    file_overhead_dict = {}
    
    # initialize the dict with task_list and overhead_type
    for pid_file, pid_stat in file_dict.items():
        file_overhead_dict[pid_file] = {}
        for task in task_list:
            file_overhead_dict[pid_file][task] = {}
            for overhead_type in overhead_dict[stat_type]:
                file_overhead_dict[pid_file][task][overhead_type] = []

    for pid_file, pid_stat in file_dict.items():
        prev_task = None
        task_pid = (pid_file.split("/")[-1]).split("-")[0]      
        for entry in pid_stat:
            if "Task" in entry.keys():
                task_time = entry["Task"]
                overhead_time_list.append(task_time)
                if stat_type == "VOL":
                    
                    if prev_task is not None:
                        for overhead_type in overhead_dict[stat_type]:
                            file_overhead_dict[pid_file][prev_task][overhead_type].append(task_time[overhead_type])
                
                elif stat_type == "VFD":
                    task_name = task_time['task_name']
                    # task_name = task_name.split("-")[0]
                    task_name = task_name.replace(f"-{task_pid}","")
                    for overhead_type in overhead_dict[stat_type]:
                        file_overhead_dict[pid_file][task_name][overhead_type].append(task_time[overhead_type])
                    # Get the other entry.keys() to get first and last task time, ther eis only 2 keys 
                    other_keys = [key for key in entry.keys() if key != "Task"]
                    other_key = other_keys[0]
                    first_file_time, last_file_time = get_first_last_file_time(entry[other_key], first_file_time, last_file_time)
            else:
                
                file_id = list(entry.keys())[0]
                first_file_time, last_file_time = get_first_last_file_time(entry[file_id], first_file_time, last_file_time)
                
                if "task_name" in entry[file_id].keys():
                    task_name = entry[file_id]['task_name']
                    # split task_name-pid to get task_name only
                    # task_name = task_name.split("-")[0]
                    task_name = task_name.replace(f"-{task_pid}","")
                    prev_task = task_name
    
     
    if workload_name == "pyflextrkr":
        task_overhead_dict = {}
        # initialize the dict with task_list and overhead_type
        for task in task_list:
            task_overhead_dict[task] = {}
            for overhead_type in overhead_dict[stat_type]:
                task_overhead_dict[task][overhead_type] = 0

        for pid_file, overhead_time_dict in file_overhead_dict.items():
            # print(f"{pid_file}: {overhead_time_dict}")
            for task in task_list:
                for overhead_type in overhead_time_dict[task]:
                    pid_overhead = sum(overhead_time_dict[task][overhead_type])
                    task_overhead = task_overhead_dict[task][overhead_type]
                    
                    # # pick only 
                    if pid_overhead > task_overhead:
                        task_overhead_dict[task][overhead_type] = pid_overhead

        # show task_overhead_dict
        for task in task_list:
            print(f"{task}: {task_overhead_dict[task]}")
    
        # Sum all overhead except VOL-Overhead(us)/VFD-Overhead(us)
        for task in task_list:
            task_overhead_dict[task]["Sum-Overhead(us)"] = 0
            for overhead_type in overhead_dict[stat_type]:
                # Pick the largest value of overhead_type
                # print(f'{task}: {overhead_type}: {overhead_time_dict[task][overhead_type]}')
                task_overhead_dict[task]["Sum-Overhead(us)"] += task_overhead_dict[task][overhead_type]
            print(f'{task}: Sum-Overhead(us): {task_overhead_dict[task]["Sum-Overhead(us)"]}')

        if workflow_time == 0:
            workflow_time = last_file_time - first_file_time # get a estimate of workflow time from first file to last file time
        print(f"Total task time (=workflow time) (us): {workflow_time}")
        
        # Sum all tasks overhead
        sum_tasks_overhead = 0
        for task in task_list: sum_tasks_overhead += task_overhead_dict[task]["Sum-Overhead(us)"]
            
    elif workload_name == "h5bench":
        sum_tasks_overhead = 0
        for pid_file, overhead_time_dict in file_overhead_dict.items():
            # print(f"{pid_file}: {overhead_time_dict}")
            for task in task_list:
                for overhead_type in overhead_time_dict[task]:
                    pid_overhead = sum(overhead_time_dict[task][overhead_type])
                    # task_overhead = task_overhead_dict[task][overhead_type]
                    sum_tasks_overhead += pid_overhead

    # Show overhead percentage on total task time
    print(f"Sum tasks overhead (us): {sum_tasks_overhead}")
    print(f"Workflow time (us): {workflow_time}")
    print(f"Overhead percentage on total task time: {sum_tasks_overhead/workflow_time*100}%")
    
    return file_overhead_dict, sum_tasks_overhead

print("VFD overhead")
WORKFLOW_TIME=8829 * 1000
vfd_overhead, _ = show_detail_overhead_old("VFD", vfd_dict, TASK_LISTS, workflow_time=WORKFLOW_TIME, workload_name=WORKLOAD_NAME) # 

print() # add a new line between VFD and VOL overhead

print("VOL overhead")
vol_overhead, _ = show_detail_overhead_old("VOL", vol_dict, TASK_LISTS, workflow_time=WORKFLOW_TIME, workload_name=WORKLOAD_NAME) 

VFD overhead
run_idfeature: {'VFD-Init(us)': 124.967, 'VFD-Term(us)': 0, 'VFD-Tracker-Init(us)': 3569.621, 'VFD-Stat-Add(us)': 42851.933, 'VFD-Stat-Update(us)': 124422.61700000001, 'VFD-Stat-Rm(us)': 0, 'VFD-Stat-Log(us)': 12281.461}
run_tracksingle: {'VFD-Init(us)': 594.3140000000001, 'VFD-Term(us)': 0, 'VFD-Tracker-Init(us)': 1993.443, 'VFD-Stat-Add(us)': 4173.067000000001, 'VFD-Stat-Update(us)': 146376.89399999997, 'VFD-Stat-Rm(us)': 0, 'VFD-Stat-Log(us)': 59464.41700000001}
run_gettracks: {'VFD-Init(us)': 693.4780000000002, 'VFD-Term(us)': 0, 'VFD-Tracker-Init(us)': 1925.571, 'VFD-Stat-Add(us)': 4049.559000000001, 'VFD-Stat-Update(us)': 112675.66300000002, 'VFD-Stat-Rm(us)': 0, 'VFD-Stat-Log(us)': 83149.20999999996}
run_trackstats: {'VFD-Init(us)': 58.544, 'VFD-Term(us)': 0, 'VFD-Tracker-Init(us)': 0, 'VFD-Stat-Add(us)': 465.66700000000003, 'VFD-Stat-Update(us)': 143020.27300000002, 'VFD-Stat-Rm(us)': 0, 'VFD-Stat-Log(us)': 14332.126}
run_identifymcs: {'VFD-Init(us)': 24.866, 'VFD-

In [75]:
def show_io_time(file_dict, task_list, workflow_time=0, workload_name="pyflextrkr"):
    iotime_param = ["POSIX-READ-Time(us)", "POSIX-WRITE-Time(us)", "POSIX-OPEN-Time(us)", "POSIX-CLOSE-Time(us)", "POSIX-DELETE-Time(us)", ]
    iotime_task_dict = {}
    
    iotime_list = []
    first_file_time = -1
    last_file_time = -1
    
    file_iotime_dict = {}
    
    # initialize the dict with task_list and overhead_type
    for pid_file, pid_stat in file_dict.items():
        file_iotime_dict[pid_file] = {}
        for task in task_list:
            file_iotime_dict[pid_file][task] = {}
            for iotime_type in iotime_param:
                file_iotime_dict[pid_file][task][iotime_type] = []

    for pid_file, pid_stat in file_dict.items():
        prev_task = None
        task_pid = (pid_file.split("/")[-1]).split("-")[0]      
        for entry in pid_stat:
            if "Task" in entry.keys():
                task_time = entry["Task"]
                iotime_list.append(task_time)
                task_name = task_time['task_name']
                # task_name = task_name.split("-")[0]
                task_name = task_name.replace(f"-{task_pid}","")
                for iotime_type in iotime_param:
                    file_iotime_dict[pid_file][task_name][iotime_type].append(task_time[iotime_type])
                # Get the other entry.keys() to get first and last task time, ther eis only 2 keys 
                other_keys = [key for key in entry.keys() if key != "Task"]
                other_key = other_keys[0]
                first_file_time, last_file_time = get_first_last_file_time(entry[other_key], first_file_time, last_file_time)
            else:
                
                file_id = list(entry.keys())[0]
                first_file_time, last_file_time = get_first_last_file_time(entry[file_id], first_file_time, last_file_time)
                
                if "task_name" in entry[file_id].keys():
                    task_name = entry[file_id]['task_name']
                    # split task_name-pid to get task_name only
                    # task_name = task_name.split("-")[0]
                    task_name = task_name.replace(f"-{task_pid}","")
                    prev_task = task_name
    
     
    if workload_name == "pyflextrkr":
        task_iotime_dict = {}
        # initialize the dict with task_list and overhead_type
        for task in task_list:
            task_iotime_dict[task] = {}
            for iotime_type in iotime_param:
                task_iotime_dict[task][iotime_type] = 0

        for pid_file, iotime_dict in file_iotime_dict.items():
            # print(f"{pid_file}: {overhead_time_dict}")
            for task in task_list:
                for iotime_type in iotime_dict[task]:
                    pid_iotime = sum(iotime_dict[task][iotime_type])
                    task_iotime= task_iotime_dict[task][iotime_type]
                    
                    # # pick only 
                    if pid_iotime > task_iotime:
                        task_iotime_dict[task][iotime_type] = pid_iotime

        # show task_overhead_dict
        for task in task_list:
            print(f"{task}: {task_iotime_dict[task]}")
        print("Task IO Time")
        print(task_iotime_dict)
        
        # Sum all task IO time
        sum_tasks_iotime = 0
        for task in task_list: 
            
            task_iotime = sum(task_iotime_dict[task].values())
            sum_tasks_iotime += task_iotime
        print(f"Sum tasks IO time (us): {sum_tasks_iotime}")
        
        # Print the percentage of IO time on total task time
        print(f"Workflow time (us): {workflow_time}")
        print(f"IO time percentage on total task time: {sum_tasks_iotime/workflow_time*100}%")
    
    return file_iotime_dict

show_io_time(vfd_dict, TASK_LISTS, workflow_time=WORKFLOW_TIME, workload_name=WORKLOAD_NAME)

run_idfeature: {'POSIX-READ-Time(us)': 101562.55600000001, 'POSIX-WRITE-Time(us)': 39621.752, 'POSIX-OPEN-Time(us)': 2259.046, 'POSIX-CLOSE-Time(us)': 56.92, 'POSIX-DELETE-Time(us)': 0}
run_tracksingle: {'POSIX-READ-Time(us)': 24515.890000000003, 'POSIX-WRITE-Time(us)': 5365.356, 'POSIX-OPEN-Time(us)': 1021.144, 'POSIX-CLOSE-Time(us)': 428.8339999999999, 'POSIX-DELETE-Time(us)': 0}
run_gettracks: {'POSIX-READ-Time(us)': 18734.123, 'POSIX-WRITE-Time(us)': 1168.287, 'POSIX-OPEN-Time(us)': 944.5599999999998, 'POSIX-CLOSE-Time(us)': 639.156, 'POSIX-DELETE-Time(us)': 0}
run_trackstats: {'POSIX-READ-Time(us)': 28126.182, 'POSIX-WRITE-Time(us)': 10699.672999999999, 'POSIX-OPEN-Time(us)': 185.95600000000002, 'POSIX-CLOSE-Time(us)': 69.765, 'POSIX-DELETE-Time(us)': 0}
run_identifymcs: {'POSIX-READ-Time(us)': 1052.253, 'POSIX-WRITE-Time(us)': 5982.652, 'POSIX-OPEN-Time(us)': 172.831, 'POSIX-CLOSE-Time(us)': 60.055, 'POSIX-DELETE-Time(us)': 0}
run_matchpf: {'POSIX-READ-Time(us)': 16868.772, 'POSI

{'example_stat/wrf_f24s9p12/339598-vfd_data_stat.json': {'run_idfeature': {'POSIX-READ-Time(us)': [44330.92,
    0.0,
    49102.101,
    0.0],
   'POSIX-WRITE-Time(us)': [19565.912, 0.0, 18781.429, 0.0],
   'POSIX-OPEN-Time(us)': [103.487, 0.0, 83.399, 0.0],
   'POSIX-CLOSE-Time(us)': [0.0, 14.92, 5.329, 30.281],
   'POSIX-DELETE-Time(us)': [0.0, 0.0, 0.0, 0.0]},
  'run_tracksingle': {'POSIX-READ-Time(us)': [1801.211,
    1220.988,
    0.0,
    1369.341,
    403.314,
    1022.887,
    693.102,
    2155.936,
    346.356,
    519.824,
    400.232,
    17.453,
    0.0,
    1534.693,
    673.507,
    0.0,
    581.567,
    590.735,
    550.167,
    520.749,
    559.885,
    553.861,
    573.601],
   'POSIX-WRITE-Time(us)': [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1421.766,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0],
   'POSIX-OPEN-Time(us)': [62.18,
    51.963,
    0.0,
    61.

In [76]:
# Check the VFD overhead dataset
OVERHEAD_DICT = {
        "VOL": ["VOL-Init(us)", "VOL-Term(us)", "VOL-Log(us)",
                "VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)",
                "VOL-FILE_LL-Add(us)", "VOL-FILE_LL-Rm(us)", "VOL-FILE_LL-Update(us)",
                "VOL-DS_LL-Add(us)", "VOL-DS_LL-Rm(us)", "VOL-DS_LL-Update(us)",
                "VOL-GRP_LL-Add(us)", "VOL-GRP_LL-Rm(us)",
                "VOL-DT_LL-Add(us)", "VOL-DT_LL-Rm(us)"],

        "VFD": ["VFD-Init(us)", "VFD-Term(us)", "VFD-Tracker-Init(us)", "VFD-Stat-Add(us)", "VFD-Stat-Update(us)", "VFD-Stat-Rm(us)", "VFD-Stat-Log(us)"]
    }

def get_overhead_data(stat_type, file_overhead_dict, task_list, is_merge_items=True):
    # Note: param is_merge_items is used in VOL to merge VOL-HT-Add(us), VOL-HT-Rm(us), VOL-HT-Search(us) to VOL-HT(us)
    task_overhead_dict = {}
    # initialize the dict with task_list and overhead_type
    for task in task_list:
        task_overhead_dict[task] = {}
        for overhead_type in OVERHEAD_DICT[stat_type]:
            task_overhead_dict[task][overhead_type] = 0
    
    for pid_file, overhead_time_dict in file_overhead_dict.items():
        # print(f"{pid_file}: {overhead_time_dict}")
        for task in task_list:
            for overhead_type in overhead_time_dict[task]:
                pid_overhead = sum(overhead_time_dict[task][overhead_type])
                task_overhead = task_overhead_dict[task][overhead_type]
                
                # pick only 
                if pid_overhead > task_overhead:
                    task_overhead_dict[task][overhead_type] = pid_overhead

    # Create overhead data dict
    overhead_data_dict = {}
    if stat_type == "VFD":
        for step in OVERHEAD_DICT[stat_type]:
            overhead_data_dict[step] = 0 
        
        for task in task_list:
            for key, value in task_overhead_dict[task].items():
                overhead_data_dict[key] += value
    elif stat_type == "VOL":
        # for VOL overhead, sum all VOL-HT as one step, same for VOL-FILE_LL, VOL-DS_LL, VOL-GRP_LL, VOL-DT_LL
        for step in OVERHEAD_DICT[stat_type]:
            if is_merge_items:
                if "VOL-HT" in step:
                    overhead_data_dict["VOL-HT(us)"] = 0
                elif "VOL-FILE_LL" in step:
                    overhead_data_dict["VOL-FILE_LL(us)"] = 0
                elif "VOL-DS_LL" in step:
                    overhead_data_dict["VOL-DS_LL(us)"] = 0
                elif "VOL-GRP_LL" in step:
                    overhead_data_dict["VOL-GRP_LL(us)"] = 0
                elif "VOL-DT_LL" in step:
                    overhead_data_dict["VOL-DT_LL(us)"] = 0
                else:
                    overhead_data_dict[step] = 0
            else:
                overhead_data_dict[step] = 0

        for task in task_list:
            for key, value in task_overhead_dict[task].items():
                if is_merge_items:
                    if "VOL-HT" in key:
                        overhead_data_dict["VOL-HT(us)"] += value
                    elif "VOL-FILE_LL" in key:
                        overhead_data_dict["VOL-FILE_LL(us)"] += value
                    elif "VOL-DS_LL" in key:
                        overhead_data_dict["VOL-DS_LL(us)"] += value
                    elif "VOL-GRP_LL" in key:
                        overhead_data_dict["VOL-GRP_LL(us)"] += value
                    elif "VOL-DT_LL" in key:
                        overhead_data_dict["VOL-DT_LL(us)"] += value
                    else:
                        overhead_data_dict[key] += value
                else:
                    overhead_data_dict[key] += value 

    return overhead_data_dict

def get_dayu_overhead_data():
    # Input_Parser = VFD-Init(us)+VFD-Tracker-Init(us)+VOL-Init(us)
    # Access_Tracker = VFD-Stat-Add(us), VFD-Stat-Rm(us),VFD-Stat-Log(us), VOL-HT-Add(us), VOL-FILE_LL-Add(us), VOL-DS_LL-Add(us), 
    # VOL-GRP_LL-Add(us), VOL-DT_LL-Add(us), VOL-FILE_LL-Rm(us), VOL-DS_LL-Rm(us), VOL-GRP_LL-Rm(us), VOL-DT_LL-Rm(us),VOL-Log(us)
    # Characteristic_Mapper = VFD-Stat-Update(us),VOL-HT-Search(us),VOL-FILE_LL-Update(us),VOL-DS_LL-Update(us)
    dayu_overhead_data = {}
    vfd_overhead_data = get_overhead_data("VFD", vfd_overhead, TASK_LISTS)
    vol_overhead_data = get_overhead_data("VOL", vol_overhead, TASK_LISTS, is_merge_items=False)

    dayu_overhead_data["Input_Parser"] = vfd_overhead_data["VFD-Init(us)"] + vfd_overhead_data["VFD-Tracker-Init(us)"] + vol_overhead_data["VOL-Init(us)"]
    dayu_overhead_data["Access_Tracker"] = vfd_overhead_data["VFD-Stat-Add(us)"] + vfd_overhead_data["VFD-Stat-Rm(us)"] + vfd_overhead_data["VFD-Stat-Log(us)"] 
    dayu_overhead_data["Access_Tracker"] += vol_overhead_data["VOL-HT-Add(us)"] + vol_overhead_data["VOL-FILE_LL-Add(us)"] + vol_overhead_data["VOL-DS_LL-Add(us)"]
    dayu_overhead_data["Access_Tracker"] += vol_overhead_data["VOL-GRP_LL-Add(us)"] + vol_overhead_data["VOL-DT_LL-Add(us)"] + vol_overhead_data["VOL-FILE_LL-Rm(us)"] 
    dayu_overhead_data["Access_Tracker"] += vol_overhead_data["VOL-DS_LL-Rm(us)"] + vol_overhead_data["VOL-GRP_LL-Rm(us)"] + vol_overhead_data["VOL-DT_LL-Rm(us)"] + vol_overhead_data["VOL-Log(us)"]
    dayu_overhead_data["Characteristic_Mapper"] = vfd_overhead_data["VFD-Stat-Update(us)"] + vol_overhead_data["VOL-HT-Search(us)"] + vol_overhead_data["VOL-FILE_LL-Update(us)"] 
    dayu_overhead_data["Characteristic_Mapper"] += vol_overhead_data["VOL-DS_LL-Update(us)"]

    return dayu_overhead_data

In [77]:
import matplotlib.pyplot as plt
import numpy as np

PLOTS_PATH = f"/Users/yejie/Projects/SCS_Lab/SC24/Candice_Paper/plots"

In [78]:
# Function to Create the Pie chart (donut chart)
def create_donut_chart(overhead_name, data, labels, colors, hatches, workload_name):
    assert overhead_name in ["VFD", "VOL", "DAYU"]
    
    labelfontsize = 12
    # Calculate the percentages
    total_time = sum(data)
    real_labels = []
    for i, label in enumerate(labels):
        pct = (data[i]/total_time) * 100
        real_labels.append(f"{label} ({pct:.2f}%)")
    print(real_labels)
    # Plot
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.pie(data, colors=colors, startangle=90, frame=False, hatch=hatches)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # Draw a circle at the center of pie to make it look like a donut
    centre_circle = plt.Circle((0,0),0.80,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    # Equal aspect ratio ensures that pie/donut is drawn as a circle.
    plt.axis('equal')  

    plt.tight_layout()
    # Place the legend inside the donut center
    legend = plt.legend(real_labels, loc='center', fontsize=labelfontsize)
    
    # Add a text box for the total time below the legend
    if workload_name == "pyflextrkr": 
        plt.figtext(0.5,  # x-position
                    0.21,  # y-position
                    f'Total Time: {total_time/1e6:.2f} s',  # text
                    ha='center',  # horizontal alignment
                    va='bottom',  # vertical alignment
                    fontsize=labelfontsize)  # font size
    else:
        plt.figtext(0.5,  # x-position
                    0.21,  # y-position
                    f'Total Time: {total_time/1e3:.2f} ms',  # text
                    ha='center',  # horizontal alignment
                    va='bottom',  # vertical alignment
                    fontsize=labelfontsize)

    if overhead_name == "VFD":
        # Saving the VFD overhead plot as a PNG file
        plt.savefig(f'{PLOTS_PATH}/{workload_name}_VFD_overhead_anatomy.png', dpi=300)  # Adjust the dpi for higher or lower resolution

        # Saving the VFD overhead plot as an EPS file
        plt.savefig(f'{PLOTS_PATH}/{workload_name}_VFD_overhead_anatomy.eps', format='eps', dpi=300)  
    elif overhead_name == "VOL":
        # Saving the VOL overhead plot as a PNG file
        plt.savefig(f'{PLOTS_PATH}/{workload_name}_VOL_overhead_anatomy.png', dpi=300)
        # Saving the VFD overhead plot as an EPS file
        plt.savefig(f'{PLOTS_PATH}/{workload_name}_VOL_overhead_anatomy.eps', format='eps', dpi=300)
    elif overhead_name == "DAYU":
        # Saving the VOL overhead plot as a PNG file
        plt.savefig(f'{PLOTS_PATH}/{workload_name}_DAYU_overhead_anatomy.png', dpi=300)
        # Saving the VFD overhead plot as an EPS file
        plt.savefig(f'{PLOTS_PATH}/{workload_name}_DAYU_overhead_anatomy.eps', format='eps', dpi=300)
    else:
        print("Invalid overhead name")


Generating Internal Anatomy Test Plots

In [79]:
# Here the vfd and vol overhead data is got from the specified folder workload folder
#   -- E.g., we use example_stat/f48s9p24 for pyflextrkr workload, and use 
# h5bench(strong_scale/sync-write-3d-contig-contig-read-full-80g_3)--in example_stat/ for h5bench workload
# 1. Plot VFD overhead Anatomy figure
# get the VFD overhead data
vfd_overhead_data=get_overhead_data("VFD", vfd_overhead, TASK_LISTS)
print(f"Workload ${WORKLOAD_NAME} VFD overhead data")
print(vfd_overhead_data)

# Preparing the VFD overhead data for plotting
vfd_steps = ['Init', 'Tracker-Init', 'Stat-Add', 'Stat-Update', 'Stat-Rm', 'Stat-Log']

vfd_steps_overhead = list(value for key, value in vfd_overhead_data.items() if "Term" not in key)

colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#c2c2f0','#ffb3e6']

hatches = ['xx', '||', 'oo', '//', '++', '..', '--']

# create_donut_chart("VFD", vfd_steps_overhead, vfd_steps, colors, hatches, workload_name=WORKLOAD_NAME)

Workload $pyflextrkr VFD overhead data
{'VFD-Init(us)': 1651.0280000000005, 'VFD-Term(us)': 0, 'VFD-Tracker-Init(us)': 7488.635, 'VFD-Stat-Add(us)': 52870.08699999999, 'VFD-Stat-Update(us)': 1107415.987, 'VFD-Stat-Rm(us)': 0, 'VFD-Stat-Log(us)': 239933.732}


In [80]:
# 2. Plot VOL overhead Anatomy figure
# get the VOL overhead data
vol_overhead_data=get_overhead_data("VOL", vol_overhead, TASK_LISTS)
print(f"Workload ${WORKLOAD_NAME} VOL overhead data")
print(vol_overhead_data)

# Preparing the VFD overhead data for plotting
vol_steps = ['Init', 'Log', 'HT', 'FILE_LL', 'DS_LL', 'GRP_LL', 'DT_LL']

vol_steps_overhead = list(value for key, value in vol_overhead_data.items() if "Term" not in key)

colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#c2c2f0','#ffb3e6', '#ff7f0e']

hatches = ['xx', '||', 'oo', '//', '++', '..', '--']

# create_donut_chart("VOL", vol_steps_overhead, vol_steps, colors, hatches, workload_name=WORKLOAD_NAME)

Workload $pyflextrkr VOL overhead data
{'VOL-Init(us)': 9967, 'VOL-Term(us)': 0, 'VOL-Log(us)': 292928, 'VOL-HT(us)': 553486, 'VOL-FILE_LL(us)': 147904, 'VOL-DS_LL(us)': 1436893, 'VOL-GRP_LL(us)': 117838, 'VOL-DT_LL(us)': 0}


In [81]:
# 3. Plot DAYU overhead Anatomy figure
# get the DAYU overhead data
dayu_overhead_data = get_dayu_overhead_data()
print("DAYU overhead data")
print(dayu_overhead_data)

# Preparing the DAYU overhead data for plotting
dayu_steps = list(dayu_overhead_data.keys())

dayu_overhead = list(dayu_overhead_data.values())

colors = ['#ff9999','#66b3ff','#99ff99']

hatches = ['xx', '||', '..']

# create_donut_chart("DAYU", dayu_overhead, dayu_steps, colors, hatches, workload_name=WORKLOAD_NAME)

DAYU overhead data
{'Input_Parser': 19106.663, 'Access_Tracker': 2649958.819, 'Characteristic_Mapper': 1299309.987}


TODO (JieYe):
- Internal dataset (Done) --> from PyFlextrkr
1. Create a Pie Graph for VFD overhead including VFD-Init(us), VFD-Term(us) items. The value of VFD-Init(us) is the sum of 9 stages using wrf-workflow(s9f48p9: posix or mmap).
2. Create a Pie Graph for VOL overhead following items. 
   "VOL": ["VOL-Init(us)", "VOL-Term(us)", "VOL-Log(us)",
                "VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)",
                "VOL-FILE_LL-Add(us)", "VOL-FILE_LL-Rm(us)", "VOL-FILE_LL-Update(us)",
                "VOL-DS_LL-Add(us)", "VOL-DS_LL-Rm(us)", "VOL-DS_LL-Update(us)",
                "VOL-GRP_LL-Add(us)", "VOL-GRP_LL-Rm(us)",
                "VOL-DT_LL-Add(us)", "VOL-DT_LL-Rm(us)"],
    The first 3 is indenependent, Sum("VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)") as a single "HT-Overhead(us)", same for the rest.

- dset_overhead: only care dset.log (compare large vs small) --- Done
  - data_array_len(1D): 1024, 4:processes, 1: first trail
  - grep "read_time" and "write_time" (average over 3 trials)
  - 1) write graph 
    - different procs for 1024(data_len), X-axis: procs num, y-xis: write-time
    - legend: varis data_len
  - 2) read graph
      - different procs for 1024(data_len), X-axis: procs num, y-xis: read-time
      - legend: varis data_len
  - 3) workflow time graph (write + read)
    - different procs for 1024(data_len), X-axis: procs num, y-xis: read-time
    - legend: varis data_len 

In [82]:
# functions to handle the dset_overhead_compare_results
def get_avg_write_read_time(datset_path, ds_type, ds_size, proc):
    trial_nums = 3
    write_times = []
    read_times = []
    for i in range(1, trial_nums+1):
        file_name = f"{datset_path}/{ds_type}_dset_{ds_size}_{proc}_{i}_run/{ds_type}_dset_{ds_size}_{proc}_{i}_run.log"
        with open(file_name, "r") as f:
            for line in f:
                if "Write time:" in line:
                    # comfirmed with Candice that the write time unit is in milliseconds not seconds
                    write_times.append(float(line.strip().split(" ")[2].strip())) # unit: milliseconds
                if "Read time:" in line:
                    read_times.append(float(line.strip().split(" ")[2].strip())) # unit: milliseconds
    
    avg_write, avg_read= sum(write_times)/3, sum(read_times)/3
    # print(f"Avg Time of {ds_type}_dset_{ds_size}_{proc}_run: Write Time = {avg_write:.2f} seconds, Read Time = {avg_read:.2f} milliseconds")
    return avg_write, avg_read 


# function to get the dataset from files based on the configurations
def get_write_read_times(DATASET_PATH, ds_type, ds_sizes, procs):
    write_times = {}
    read_times = {}
    # initialize the dict with ds_sizes as the keys
    for ds_size in ds_sizes:
        write_times[f'{ds_size}'] = []
        read_times[f'{ds_size}'] = []

    if ds_type == "small":
        for ds_size in ds_sizes:
            for proc in procs:
                avg_write, avg_read = get_avg_write_read_time(DATASET_PATH, "small", ds_size, proc)
                write_times[f'{ds_size}'].append(avg_write)
                read_times[f'{ds_size}'].append(avg_read)
    elif ds_type == "large":
        for proc in procs:
            for ds_size in ds_sizes:
                avg_write, avg_read = get_avg_write_read_time(DATASET_PATH, "large", ds_size, proc)
                write_times[f'{ds_size}'].append(avg_write)
                read_times[f'{ds_size}'].append(avg_read)
    else:
        print("Invalid dataset type")
        return None, None
    
    return write_times, read_times

# function to get the workflow time, which is the sum of the write and read times in each workloads
def get_workflow_times(write_times, read_times):
    workflow_times = {}
    for ds_size in write_times.keys():
        workflow_times[f'{ds_size}'] = [w + r for w,r in zip(write_times[ds_size], read_times[ds_size])]
    return workflow_times

In [83]:
DATASET_PATH=f"/Users/yejie/Projects/SCS_Lab/SC24/Candice_Paper/candice_data/dset_overhead_compare_results"
PROCS=[4, 8, 16, 32]
DS_SIZES=[1024, 2048, 4096, 8192]


# 3. Get the Write and Read time of small dataset
small_write_times, small_read_times = get_write_read_times(DATASET_PATH, "small", DS_SIZES, PROCS)
small_workflow_time = get_workflow_times(small_write_times, small_read_times)

# 4. Get the Write and Read time of large dataset
large_write_times, large_read_times = get_write_read_times(DATASET_PATH, "large", DS_SIZES, PROCS)
large_workflow_time = get_workflow_times(large_write_times, large_read_times)

print("Average Write, Read, and Workflow(=write+read) Times for small dataset")
print(f"Write Times: {small_write_times}")
print(f"Read Times: {small_read_times}")
print(f"Workflow Times: {small_workflow_time}")

print()

print("Average Write, Read, and Workflow(=write+read) Times for large dataset")
print(f"Write Times: {large_write_times}")
print(f"Read Times: {large_read_times}")
print(f"Workflow Times: {large_workflow_time}")

#  TODO: Plotting the responding graphs
# Plot the Write graph


# Plot the Read graph



FileNotFoundError: [Errno 2] No such file or directory: '/Users/yejie/Projects/SCS_Lab/SC24/Candice_Paper/candice_data/dset_overhead_compare_results/small_dset_1024_4_1_run/small_dset_1024_4_1_run.log'

ToDo(JieYe):
- h5bench test graph
  - Internal test pie graph (Showing VFD overhead and VOL overhead based on strong_scale/sync-write-3d-contig-contig-read-full-80g_3)
    - Using Internal test scripts.
  - Strong_Scale figure
    - 1) write vs read graph (delete)
      - x-axis: [1d, 2d, 3d(20G), 3d(40G), 3d(80G)]
      - y-axis: time (workflow time = write + read)
      - In the log file, first runtime is write time, 2nd one is read time. ===> workflow_time=write + read
    - 2) Overhead graph (VFD & VOL) ---- sum ---> get average value of (overhead) and workflowtime
      - x-axis: [1d, 2d, 3d(20G), 3d(40G), 3d(80G)]
      - y-axis: overhead (total overhead_time / workflowtime(write + read))
      - legend: (VFD, VOL)
  - Weak_Scale figure
    - 1) Overhead graph (VFD & VOL) ---- sum
      - x-axis: [1, 4, 16, 64] ==> 1 is from strong scale 1D
      - y-axis: overhead (total overhead_time / workflowtime(write + read))
      - legend: (VFD, VOL)
      - QA: There are multiple logs due to multiple processes. 
        - A. How do you calculate the worflowtime for multiple processes?
            - Select the max value from each trial, then get average for 3 trials???
        - B. How do you calculate the VFD & VOL overhead for multiple processes? 
          - Based on current method, they have the same taskname, you will sum the overhead for multiple processes. Is it right?
  - There is one more graph I forgot to mention that I also wanted your help to draw, is to draw the same anatomy graph using the json data in the folder strong_scale/sync-write-3d-contig-contig-read-full-80g_3

Meng Tang (Candice): So I there are total 4 anatomy graphs, 2 from PyFlextrkr, 2 from h5bench.

In [ ]:
def get_workflow_time_from_log(log_file, workload_name="pyflextrkr"):
    workflow_time = 0
    if workload_name == "pyflextrkr":
        TIME_STR_START = "Pyflextrkr TIME:"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    workflow_time = float(line.split(" ")[2])
                    # Convert from seconds to microseconds
                    workflow_time = workflow_time * 1000000
                    break
    elif workload_name == "h5bench": 
        TIME_STR_START = "Runtime:"
        with open(log_file, "r") as f:
            for line in f:
                if TIME_STR_START in line:
                    runtime = float(line.split(" ")[7])
                    print(f"runtime = {runtime}")
                    # Convert from seconds to microseconds
                    workflow_time += runtime * 1000000
    else:
        print("WORKLOAD_NAME not recognized")

    return workflow_time

def get_avg_workflow_time(dataset_prefix_name, workload_name="pyflextrkr", proc_num=1):
    trial_nums = 3
    workflow_times = []
    for i in range(1, trial_nums+1):
        stat_path = f"{dataset_prefix_name}_{i}"
        if proc_num == 1:
            workflow_time = get_workflow_time(stat_path, workload_name=workload_name)
        else:
            max_workflow_time = 0
            for id in range(1, proc_num + 1):
                file_name = f"{stat_path}/sync-write-1d-contig-contig-read-full-1p-{id}-h5bench.log"
                workflow_t = get_workflow_time_from_log(file_name, workload_name=workload_name)
                ## get the max workflow time of multiple processes
                # if workflow_t > max_workflow_time:
                #     max_workflow_time = workflow_t
                ## get the sum workflow time of multiple processes
                max_workflow_time += workflow_t

            workflow_time = max_workflow_time
        
        workflow_times.append(workflow_time)
    
    avg_workflow = sum(workflow_times)/trial_nums
    return avg_workflow

def get_avg_vfd_vol_overhead(dataset_prefix_name, TASK_LISTS, workload_n="pyflextrkr"):
    trial_nums = 3
    vfd_overheads = []
    vol_overheads = []
    for i in range(1, trial_nums+1):
        base_stat_path = f"{dataset_prefix_name}_{i}"
        vfd_files = sload.find_files_with_pattern(base_stat_path, "vfd") 
        print(f"vfd_files: {vfd_files}")
        vfd_dict = sload.load_stat_json(vfd_files)
        print("loading vfd json done")
        vol_files = sload.find_files_with_pattern(base_stat_path, "vol") 
        print(f"vol_files: {vol_files}")
        vol_dict = sload.load_stat_json(vol_files)
        print("loading vol json done")
        print("Handle VFD and VOL overhead")
        workflow_t = get_workflow_time(base_stat_path, workload_name=workload_n)
        _, sum_vfd_overhead = show_detail_overhead_old("VFD", vfd_dict, TASK_LISTS, workflow_time=workflow_t, workload_name=workload_n) #
        _, sum_vol_overhead = show_detail_overhead_old("VOL", vol_dict, TASK_LISTS, workflow_time=workflow_t, workload_name=workload_n) #
        vfd_overheads.append(sum_vfd_overhead)
        vol_overheads.append(sum_vol_overhead)
    
    avg_vfd_overhead = sum(vfd_overheads)/trial_nums
    avg_vol_overhead = sum(vol_overheads)/trial_nums
    return avg_vfd_overhead, avg_vol_overhead

# functions to get the vfd and vol overheads (%) via increasing the scalce
def get_h5bench_strong_scale_overheads(base_path, ds_dim, ds_sizes_3d):
    overheads = {}
    overheads_times = {}
    for dim in ds_dim:
        if dim == 3:
            for size in ds_sizes_3d:
                stat_path = f"{base_path}/strong_scale/sync-write-{dim}d-contig-contig-read-full-{size}g"
                #  get the average workflow time of the dataset
                avg_workflow_time = get_avg_workflow_time(stat_path, workload_name="h5bench")
                #  get the avergae overhead time of the dataset
                TASK_LISTS = [f'sync-write-{dim}d-contig-contig-read-full-{size}g']
                avg_vfd_overhead, avg_vol_overhead = get_avg_vfd_vol_overhead(stat_path, TASK_LISTS, workload_n="h5bench")

                overheads[f'{dim}D-{size}G'] = {'vfd_overhead(%)': (avg_vfd_overhead/avg_workflow_time*100), 'vol_overhead(%)': avg_vol_overhead/avg_workflow_time*100}
                overheads_times[f'{dim}D-{size}G'] = {'vfd_overhead(us)': avg_vfd_overhead, 'vol_overhead(us)': avg_vol_overhead, 'workflow_time(us)': avg_workflow_time}
        else:
            stat_path = f"{base_path}/strong_scale/sync-write-{dim}d-contig-contig-read-full"
            TASK_LISTS = [f'sync-write-{dim}d-contig-contig-read-full']
            #  get the average workflow time of the dataset
            avg_workflow_time = get_avg_workflow_time(stat_path, workload_name="h5bench")
            #  get the avergae overhead time of the dataset
            avg_vfd_overhead, avg_vol_overhead = get_avg_vfd_vol_overhead(stat_path, TASK_LISTS, workload_n="h5bench")

            overheads[f'{dim}D'] = {'vfd_overhead(%)': (avg_vfd_overhead/avg_workflow_time*100), 'vol_overhead(%)': (avg_vol_overhead/avg_workflow_time*100)}
            overheads_times[f'{dim}D'] = {'vfd_overhead(us)': avg_vfd_overhead, 'vol_overhead(us)': avg_vol_overhead, 'workflow_time(us)': avg_workflow_time}
    
    return overheads, overheads_times


In [ ]:
# H5BENCH_DT_PATH=f"/Users/yejie/Projects/SCS_Lab/SC24/Candice_Paper/candice_data/h5bench"
H5BENCH_DT_PATH="/Users/mengtang/script/dayu-tracker/flow_analysis/h5bench/"

DS_DIM=[1, 2, 3]
DS_SIZE_3D=[20, 40, 80]

# get the overheads of the strong scale
strong_overheads, strong_verheads_times = get_h5bench_strong_scale_overheads(H5BENCH_DT_PATH, DS_DIM, DS_SIZE_3D)

print("*************************************************************************************************************")
print("Strong Scale Overheads")
print(strong_overheads)

print()

print("Strong Scale Overheads Times")
print(strong_verheads_times)

# ToDo: Plot the overheads of the strong scale
# H5BENCH_DT_PATH="/Users/mengtang/script/dayu-tracker/flow_analysis/example_stat/strong_scale"

NameError: name 'get_workflow_time' is not defined

In [ ]:
# get the overheads of the weak scale
def get_avg_vfd_vol_overhead_mutiprocs(dataset_prefix_name, op_types, TASK_LISTS, workload_n="pyflextrkr", proc_num=1):
    # This function may have some problems, needing to confirm with Candice
    trial_nums = 3
    vfd_overheads = []
    vol_overheads = []
    for i in range(1, trial_nums+1):
        base_stat_path = f"{dataset_prefix_name}_{i}"
        # get the workflow time for multi-procs (select the max value)
        max_workflow_t = 0
        for id in range(1, proc_num + 1):
            workflow_time = get_workflow_time_from_log(f"{base_stat_path}/sync-write-1d-contig-contig-read-full-1p-{id}-h5bench.log", workload_name=workload_n)
            ## get the max workflow time of multiple processes
            # if workflow_time > max_workflow_t:
            #     max_workflow_t = workflow_time
            ## get the sum workflow time of multiple processes
            max_workflow_t += workflow_time
        
        sum_vfd_overhead = 0
        sum_vol_overhead = 0
        for op_type in op_types:
            if op_type == "write":
                write_base_stat_path = f"{base_stat_path}/write"
                vfd_files = sload.find_files_with_pattern(write_base_stat_path, "vfd") 
                print(f"vfd_files for write operation: {vfd_files}")
                vfd_dict = sload.load_stat_json(vfd_files)
                print("loading vfd write json done")
                vol_files = sload.find_files_with_pattern(write_base_stat_path, "vol") 
                print(f"vol_files for write operation: {vol_files}")
                vol_dict = sload.load_stat_json(vol_files)
                print("loading vol write json done")
                print("Handle VFD and VOL overhead")
                _, sum_write_vfd_overhead = show_detail_overhead_old("VFD", vfd_dict, TASK_LISTS, workflow_time=max_workflow_t, workload_name=workload_n) #
                _, sum_write_vol_overhead = show_detail_overhead_old("VOL", vol_dict, TASK_LISTS, workflow_time=max_workflow_t, workload_name=workload_n) #
                ## get the average overheads
                # sum_vfd_overhead += sum_write_vfd_overhead / proc_num
                # sum_vol_overhead += sum_write_vol_overhead / proc_num
                ## get the sum overheads
                sum_vfd_overhead += sum_write_vfd_overhead
                sum_vol_overhead += sum_write_vol_overhead
            else:
                # read operation
                read_base_stat_path = f"{base_stat_path}/read"
                print(f"base_stat_path for read operation: {read_base_stat_path}")
                vfd_files = sload.find_files_with_pattern(read_base_stat_path, "vfd") 
                print(f"vfd_files for read operation: {vfd_files}")
                vfd_dict = sload.load_stat_json(vfd_files)
                print("loading vfd read json done")
                vol_files = sload.find_files_with_pattern(read_base_stat_path, "vol") 
                print(f"vol_files for read operation: {vol_files}")
                vol_dict = sload.load_stat_json(vol_files)
                print("loading vol read json done")
                print("Handle VFD and VOL overhead")
                _, sum_read_vfd_overhead = show_detail_overhead_old("VFD", vfd_dict, TASK_LISTS, workflow_time=max_workflow_t, workload_name=workload_n) #
                _, sum_read_vol_overhead = show_detail_overhead_old("VOL", vol_dict, TASK_LISTS, workflow_time=max_workflow_t, workload_name=workload_n) #
                ## get the average overheads
                # sum_vfd_overhead += sum_read_vfd_overhead / proc_num
                # sum_vol_overhead += sum_read_vol_overhead / proc_num
                ## get the sum overheads
                sum_vfd_overhead += sum_read_vfd_overhead
                sum_vol_overhead += sum_read_vol_overhead
                print(f"sum_vfd_overhead (read): {sum_vfd_overhead}, sum_vol_overhead (read): {sum_vol_overhead}")
            
        vfd_overheads.append(sum_vfd_overhead)
        vol_overheads.append(sum_vol_overhead)
    
    avg_vfd_overhead = sum(vfd_overheads)/trial_nums
    avg_vol_overhead = sum(vol_overheads)/trial_nums
    return avg_vfd_overhead, avg_vol_overhead


def get_h5bench_weak_scale_overheads(base_path, proc_num_lst):
    overheads = {}
    overheads_times = {}
    for proc_num in proc_num_lst:
        if proc_num == 1:
            stat_path = f"{base_path}/strong_scale/sync-write-1d-contig-contig-read-full"
            TASK_LISTS = [f'sync-write-1d-contig-contig-read-full']
            #  get the average workflow time of the dataset
            avg_workflow_time = get_avg_workflow_time(stat_path, workload_name="h5bench", proc_num=proc_num)
            #  get the avergae overhead time of the dataset
            avg_vfd_overhead, avg_vol_overhead = get_avg_vfd_vol_overhead(stat_path, TASK_LISTS, workload_n="h5bench")

            overheads[f'{proc_num}'] = {'vfd_overhead(%)': (avg_vfd_overhead/avg_workflow_time*100), 'vol_overhead(%)': (avg_vol_overhead/avg_workflow_time*100)}
            overheads_times[f'{proc_num}'] = {'vfd_overhead(us)': avg_vfd_overhead, 'vol_overhead(us)': avg_vol_overhead, 'workflow_time(us)': avg_workflow_time}
        else:
            stat_path = f"{base_path}/weak_scale/p{proc_num}-sync-write-1d-contig-contig-read-full-1p"
            TASK_LISTS = [f'sync-write-1d-contig-contig-read-full-1p']
            #  get the average workflow time of the dataset
            avg_workflow_time = get_avg_workflow_time(stat_path, workload_name="h5bench", proc_num=proc_num)
            #  get the avergae overhead time of the dataset
            avg_vfd_overhead, avg_vol_overhead = get_avg_vfd_vol_overhead_mutiprocs(stat_path, ["write", "read"], TASK_LISTS, workload_n="h5bench", proc_num=proc_num)

            overheads[f'{proc_num}'] = {'vfd_overhead(%)': (avg_vfd_overhead/avg_workflow_time*100), 'vol_overhead(%)': (avg_vol_overhead/avg_workflow_time*100)}
            overheads_times[f'{proc_num}'] = {'vfd_overhead(us)': avg_vfd_overhead, 'vol_overhead(us)': avg_vol_overhead, 'workflow_time(us)': avg_workflow_time}


    return overheads, overheads_times

In [ ]:
H5BENCH_DT_PATH=f"/Users/yejie/Projects/SCS_Lab/SC24/Candice_Paper/candice_data/h5bench"
PROC_NUM_LST=[1, 4, 16, 64]

# get the overheads of the weak scale
weak_overheads, weak_verheads_times = get_h5bench_weak_scale_overheads(H5BENCH_DT_PATH, PROC_NUM_LST)

print("*************************************************************************************************************")
print("Weak Scale Overheads")
print(weak_overheads)

print()

print("Weak Scale Overheads Times")
print(weak_verheads_times)

# ToDo: Plot the overheads of the weak scale